In [26]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import re

# 1. 데이터 읽기
dong_gdf = gpd.read_file('src/dong_boundary.shp')
parks_df = pd.read_csv('src/RF_parksdf_holi_zone.csv', encoding='utf-8')
dong_df = pd.read_csv('src/dongParks_Holiday_zone_0.csv', encoding='utf-8')

parks_gdf = gpd.GeoDataFrame(parks_df, geometry=gpd.points_from_xy(parks_df['lon'], parks_df['lat']))
parks_gdf.set_crs(epsg=4326, inplace=True)  

parks_gdf = parks_gdf.to_crs(dong_gdf.crs)
parksCombined_gdf = gpd.sjoin(parks_gdf, dong_gdf)

# dong_gdf에서 온 index_right, base_date, adm_cd 칼럼 제거하고 adm_nm(행정동명)만 남김
parksAddDong_gdf = parksCombined_gdf.drop(["index_right","BASE_DATE",'ADM_CD'], axis=1)

# df.rename(columns={'old_column': 'new_column'})
parksAddDong_gdf = parksAddDong_gdf.rename(columns = {'ADM_NM':'dong'})
#parksAddDong_gdf = parksAddDong_gdf.drop(["address"], axis=1)

parksAddDong_gdf





,address,parking_type,operation_type,total_parking_spaces,base_parking_fee,base_parking_time,additional_unit_fee,additional_unit_time,fee1H,cctv_r300,...,is_open_at_17,is_open_at_18,is_open_at_19,is_open_at_20,is_open_at_21,is_open_at_22,is_open_at_23,zoning_encoded,geometry,dong
0,강남구 개포동 126-2,NW,1,132.0,200.0,5.0,200.0,5.0,2400.0,0,...,1,1,1,1,1,1,1,6,POINT (961669.167 1942092.064),개포1동
1,강남구 개포동 1266-0,NW,1,97.0,200.0,5.0,200.0,5.0,2400.0,0,...,1,1,1,1,1,1,1,10,POINT (960057.034 1942569.311),개포4동
2,강남구 개포동 1273-0,NW,1,194.0,200.0,5.0,200.0,5.0,2400.0,3,...,1,1,1,1,1,1,1,10,POINT (960420.553 1941918.468),개포4동
3,강남구 개포동 13-2,NW,1,168.0,100.0,5.0,100.0,5.0,1200.0,2,...,1,1,1,1,1,1,1,10,POINT (962812.275 1944047.886),개포3동
4,강남구 개포동 183-2,NW,1,149.0,200.0,5.0,200.0,5.0,2400.0,2,...,1,1,1,1,1,1,1,10,POINT (961654.312 1943448.697),개포2동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1620,중랑구 중화동 29-2,NW,1,130.0,100.0,5.0,100.0,5.0,1200.0,3,...,1,1,1,1,1,1,1,10,POINT (963072.124 1956004.945),중화1동
1621,중랑구 중화동 304-36,NS,3,4.0,250.0,5.0,250.0,5.0,3000.0,1,...,1,1,1,1,1,1,1,10,POINT (962792.446 1955654.654),중화2동
1622,중랑구 중화동 61-2,NS,1,1.0,100.0,5.0,100.0,5.0,1200.0,2,...,1,1,1,1,1,1,1,10,POINT (962232.367 1955166.54),중화2동
1623,중랑구 중화동 61-4,NS,3,1.0,600.0,60.0,300.0,30.0,600.0,1,...,1,1,1,1,1,1,1,10,POINT (962125.196 1955306.518),중화2동


In [27]:
is_open_cols = [f'is_open_at_{h}' for h in range(24)]
missing_cols = [col for col in is_open_cols if col not in parksAddDong_gdf.columns]
if missing_cols:
    print(f"경고: 누락된 is_open_hour 컬럼: {missing_cols}")

# 2. 동별로 is_open_hour 평균 계산 & 0.5 기준 이진화
dong_open_mean = parksAddDong_gdf.groupby('dong')[is_open_cols].mean()
dong_open_bin = (dong_open_mean >= 0.5).astype(int).reset_index()

# 4. 병합
result = pd.merge(dong_df, dong_open_bin, on='dong', how='left')

# 5. 저장
result.to_csv('src/dongParks_Holiday_zone.csv', index=False, encoding='utf-8')



In [28]:
import pandas as pd

# 두 csv 파일 읽기
df1 = pd.read_csv('src/dongParks_Holiday_zone.csv', encoding='utf-8')
df2 = pd.read_csv('src/dongParks_Holiday_zone_0.csv', encoding='utf-8')

# 첫 번째 열 이름 자동 추출
col1 = df1.columns[0]
col2 = df2.columns[0]

# 첫 번째 열의 값 비교 (순서까지 포함, 완벽히 일치 여부)
same = (df1[col1].values == df2[col2].values).all()

if same:
    print("두 csv의 첫 번째 열 값이 모두 순서까지 일치합니다.")
else:
    print("두 csv의 첫 번째 열 값이 모두 일치하지 않습니다.")


두 csv의 첫 번째 열 값이 모두 순서까지 일치합니다.


In [29]:
print(df1.columns)
print(df2.columns)

Index(['dong', 'parking_type', 'operation_type', 'total_parking_spaces',
       'Holiday_paid', 'Holiday_start', 'Holiday_end', 'base_parking_fee',
       'base_parking_time', 'additional_unit_fee', 'additional_unit_time',
       'fee1H', 'Holiday_operatingHours', 'cctv_r300', 'bus_r300',
       'complaints_r300_Holiday', 'complaint_level', 'zoning_encoded',
       'is_open_at_0', 'is_open_at_1', 'is_open_at_2', 'is_open_at_3',
       'is_open_at_4', 'is_open_at_5', 'is_open_at_6', 'is_open_at_7',
       'is_open_at_8', 'is_open_at_9', 'is_open_at_10', 'is_open_at_11',
       'is_open_at_12', 'is_open_at_13', 'is_open_at_14', 'is_open_at_15',
       'is_open_at_16', 'is_open_at_17', 'is_open_at_18', 'is_open_at_19',
       'is_open_at_20', 'is_open_at_21', 'is_open_at_22', 'is_open_at_23'],
      dtype='object')
Index(['dong', 'parking_type', 'operation_type', 'total_parking_spaces',
       'Holiday_paid', 'Holiday_start', 'Holiday_end', 'base_parking_fee',
       'base_parking_time', 